# Association mining to find common items purchased based on online retail data

Consider each invoice as a transaction in the dataset ‘D1.csv’ and build an association mining model on this dataset to identify what are the common items that customers have purchased. The task is to conduct Association mining on this data set.

Answer the following questions concerning this data and analysis.

What pre-processing was required on the dataset before building the association mining model? What variables did you include in the analysis? Justify your choice.
Conduct association mining and answer the following:
What ‘min_support’ and `min_confidence’ thresholds were set for this mining exercise? What is the rationale for why these values were chosen?
Report the top-5 rules and interpret them.
Identify top-5 common items that customers have purchased along with ‘LUNCH BAG RED RETROSPOT’.
Can you perform sequence analysis on this dataset? If yes, present your results. If not, rationalise why.
How can the outcome of this study be used by the relevant decision-makers?

In [ ]:
#Association mining to find common items purchased based on online retail data
import pandas as pd
from apyori import apriori
import seaborn as sns
import scipy.stats as stats
import statistics
import numpy as np

# load the retaildataset
df = pd.read_csv('D1.csv')
print(df.info())

# To ignore future warnings
import warnings
warnings.filterwarnings('ignore')

In [61]:
#DATATYPE CHANGES
# Convert the 'DateColumn' to a datetime data type
#df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# Convert Customer ID to object
#df['CustomerID'] = df['CustomerID'].astype(object)
# info
print(df.info())
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19663 entries, 0 to 19662
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    19663 non-null  int64  
 1   StockCode    19663 non-null  object 
 2   Description  19075 non-null  object 
 3   Quantity     19663 non-null  int64  
 4   InvoiceDate  19663 non-null  object 
 5   UnitPrice    19663 non-null  float64
 6   CustomerID   15678 non-null  float64
 7   Country      19663 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ MB
None
   InvoiceNo StockCode                         Description  Quantity  \
0     536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1     536367     84879       ASSORTED COLOUR BIRD ORNAMENT        32   
2     536373    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
3     536375    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
4     536378     22386             JUMBO BAG 

In [66]:
#DATA PEPROCESSING
#Filtering stock where untit price is less than 0
df1 = df[df["UnitPrice"] > 0]
#Filtering stock where qunatity is less than 0, products bought and returned
df1 = df1[df["Quantity"] > 0]
#Dropping all values ins stock that are nor related to real products
df1= df1[~df1['StockCode'].str.contains('DOT')]   
df1= df1[~df1['StockCode'].str.contains('gift')]   
# Stripping extra spaces in the description 
df1['Description'] = df1['Description'].str.strip() 
df1.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
df1['InvoiceNo'] = df1['InvoiceNo'].astype('str') 
# Dropping all transactions which were done on credit 
df1= df1[~df1['InvoiceNo'].str.contains('C')]    
# Inconsistent Data
df1['StockCode'] = df1['StockCode'].replace('85123a', '85123A')

print(df1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 19065 entries, 0 to 19662
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    19065 non-null  object 
 1   StockCode    19065 non-null  object 
 2   Description  19065 non-null  object 
 3   Quantity     19065 non-null  int64  
 4   InvoiceDate  19065 non-null  object 
 5   UnitPrice    19065 non-null  float64
 6   CustomerID   15676 non-null  float64
 7   Country      19065 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 1.3+ MB
None


In [54]:
print(df1['StockCode'].unique())

['85123A' '84879' '22386' '20725' '21212' '22457' '22469' '85099B' '20727'
 '22423' '47566' '22720']


In [55]:
#ASSOCIATION MINING
#Grouping data with relevent columns
transactions = df1.groupby(['InvoiceNo'])['StockCode'].apply(list)
print(transactions.head(5))
print(transactions)

InvoiceNo
536365                 [85123A]
536367                  [84879]
536373                 [85123A]
536375                 [85123A]
536378    [22386, 20725, 21212]
Name: StockCode, dtype: object
InvoiceNo
536365                  [85123A]
536367                   [84879]
536373                  [85123A]
536375                  [85123A]
536378     [22386, 20725, 21212]
                   ...          
581498    [22386, 22457, 85099B]
581538     [22469, 20727, 20725]
581579                   [22386]
581583                   [20725]
581585                   [84879]
Name: StockCode, Length: 9563, dtype: object


In [56]:
#min support is 20% and min confidence is 50%
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.02, confidence=0.5))
# print first 5 rules
print(results[:5])


[RelationRecord(items=frozenset({'20725'}), support=0.16354700407821812, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20725'}), confidence=0.16354700407821812, lift=1.0)]), RelationRecord(items=frozenset({'20727'}), support=0.13311722262888215, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20727'}), confidence=0.13311722262888215, lift=1.0)]), RelationRecord(items=frozenset({'21212'}), support=0.13803199832688487, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'21212'}), confidence=0.13803199832688487, lift=1.0)]), RelationRecord(items=frozenset({'22386'}), support=0.12736588936526194, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22386'}), confidence=0.12736588936526194, lift=1.0)]), RelationRecord(items=frozenset({'22423'}), support=0.20788455505594478, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'2242

In [57]:

def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(result_df.head(20))

   Left_side   Right_side   Support  Confidence      Lift
0                   20725  0.163547    0.163547  1.000000
1                   20727  0.133117    0.133117  1.000000
2                   21212  0.138032    0.138032  1.000000
3                   22386  0.127366    0.127366  1.000000
4                   22423  0.207885    0.207885  1.000000
5                   22457  0.130608    0.130608  1.000000
6                   22469  0.125588    0.125588  1.000000
7                   22720  0.144829    0.144829  1.000000
8                   47566  0.176200    0.176200  1.000000
9                   84879  0.152149    0.152149  1.000000
10                 85099B  0.218446    0.218446  1.000000
11                 85123A  0.235909    0.235909  1.000000
12            20725,20727  0.067029    0.067029  1.000000
13     20725        20727  0.067029    0.409847  3.078839
14     20727        20725  0.067029    0.503535  3.078839
15            20725,21212  0.036181    0.036181  1.000000
16     20725  

In [58]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(5))

        Left_side    Right_side   Support  Confidence      Lift
160         20725  85099B,20727  0.024574    0.150256  3.821531
165  85099B,20727         20725  0.024574    0.625000  3.821531
170   20725,22386        85099B  0.026038    0.770898  3.529007
169        85099B   20725,22386  0.026038    0.119196  3.529007
171  85099B,20725         22386  0.026038    0.430052  3.376507


In [59]:
from apyori import apriori

# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.05))

# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'20725'}), support=0.16354700407821812, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20725'}), confidence=0.16354700407821812, lift=1.0)]), RelationRecord(items=frozenset({'20727'}), support=0.13311722262888215, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20727'}), confidence=0.13311722262888215, lift=1.0)]), RelationRecord(items=frozenset({'21212'}), support=0.13803199832688487, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'21212'}), confidence=0.13803199832688487, lift=1.0)]), RelationRecord(items=frozenset({'22386'}), support=0.12736588936526194, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22386'}), confidence=0.12736588936526194, lift=1.0)]), RelationRecord(items=frozenset({'22423'}), support=0.20788455505594478, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'2242

In [64]:
#common products with ‘LUNCH BAG RED RETROSPOT’
df_lunch = df[df["Description"]== 'LUNCH BAG RED RETROSPOT']
print(df_lunch.head(5))

product = '20725'
df_lunchbag = result_df[result_df['Left_side'] == product]
print(df_lunchbag.info())
df_lunchbag= df_lunchbag.sort_values(by='Lift', ascending=False)
print(df_lunchbag.head(5))

    InvoiceNo StockCode              Description  Quantity      InvoiceDate  \
5      536378     20725  LUNCH BAG RED RETROSPOT        10   1/12/2010 9:37   
19     536401     20725  LUNCH BAG RED RETROSPOT         1  1/12/2010 11:21   
28     536412     20725  LUNCH BAG RED RETROSPOT         1  1/12/2010 11:49   
65     536544     20725  LUNCH BAG RED RETROSPOT         4  1/12/2010 14:32   
83     536577     20725  LUNCH BAG RED RETROSPOT        60  1/12/2010 16:13   

    UnitPrice  CustomerID         Country  
5        1.65     14688.0  United Kingdom  
19       1.65     15862.0  United Kingdom  
28       1.65     17920.0  United Kingdom  
65       4.21         NaN  United Kingdom  
83       1.45     13777.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 160 to 22
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Left_side   12 non-null     object 
 1   Right_side  12 non-null     object

# SEQUENTIAL ANALYSIS

In [52]:
#sequential analysis
transactions = df1.groupby(['InvoiceNo'])['StockCode'].apply(list)
sequences = transactions.values.tolist()
# show the first 5 sequences
print(sequences)
# Open the file in write mode ('w')
with open('seq_rule_output2.txt', 'w') as f:
    # Nothing needs to be written for it to be empty
    pass


[['85123A'], ['84879'], ['85123A'], ['85123A'], ['22386', '20725', '21212'], ['22457', '22469'], ['85099B'], ['22469'], ['85123A', '85099B'], ['84879'], ['85123A'], ['84879', '21212'], ['85123A'], ['85123A', '20725'], ['22469', '21212', '20727'], ['85123A'], ['84879', '21212'], ['85099B'], ['20727', '20725'], ['22469', '21212'], ['84879'], ['22386', '85099B', '84879', '21212', '21212'], ['22423'], ['84879', '85123A', '22423'], ['85123A', '22469', '22469'], ['85099B'], ['22423'], ['85099B'], ['22423'], ['22423'], ['21212'], ['84879'], ['22423'], ['22457', '22469'], ['22423'], ['20727', '22386', '85099B', '85123A'], ['22423', '22457', '22469', '85099B', '85123A', '20725', '21212'], ['22423'], ['22457'], ['22469'], ['85099B', '85123A'], ['20727', '85099B', '85123A'], ['20725'], ['20727'], ['85099B'], ['85123A', '22457'], ['22386'], ['20725', '20727', '21212', '22386', '22423', '22469', '84879', '85099B', '85123A'], ['85123A'], ['85123A'], ['85123A'], ['22457'], ['22386', '85099B', '84879'

In [44]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf (1).jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', supp_param, conf_param], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [45]:
get_association_rules(sequences, 0.1, 0.1)


FileNotFoundError: [Errno 2] No such file or directory: 'seq_rule_output.txt'